In [9]:
import os
os.getcwd()

'/Users/fereshteh/Documents/MyGithub/RAG/rag-eval-medical-manual/notebooks'

In [11]:
import pandas as pd

paths = {
    3: "../results/eval_results_k3.csv",
    5: "../results/eval_results_k5.csv",
    10: "../results/eval_results_k10.csv",
}

dfs = {k: pd.read_csv(p) for k, p in paths.items()}

for k, df in dfs.items():
    print(f"TOP_K={k} | rows={len(df)} | mean latency={df['latency_s'].mean():.2f}s")


TOP_K=3 | rows=10 | mean latency=21.23s
TOP_K=5 | rows=10 | mean latency=32.10s
TOP_K=10 | rows=10 | mean latency=44.60s


In [23]:
import numpy as np

def to_num(s):
    return pd.to_numeric(s, errors="coerce")

for k, df in dfs.items():
    df["correctness_num"] = to_num(df.get("correctness"))
    df["faithfulness_num"] = to_num(df.get("faithfulness"))
    dfs[k] = df



In [24]:
summary = []
for k, df in dfs.items():
    c = df["correctness_num"]
    f = df["faithfulness_num"]

    summary.append({
        "top_k": k,
        "mean_latency_s": round(df["latency_s"].mean(), 2),
        "p95_latency_s": round(df["latency_s"].quantile(0.95), 2),
        "correctness_mean": round(c.mean(), 3),
        "faithfulness_mean": round(f.mean(), 3),
        "correctness_n": int(c.notna().sum()),
        "faithfulness_n": int(f.notna().sum()),
    })

summary_df = pd.DataFrame(summary).sort_values("top_k")
summary_df


,top_k,mean_latency_s,p95_latency_s,correctness_mean,faithfulness_mean,correctness_n,faithfulness_n
0,3,21.23,28.20,1.70,1.0,10,10
1,5,32.10,41.78,2.05,1.0,10,10
2,10,44.60,58.58,2.00,0.9,10,10


In [25]:
# higher is better for correctness/faithfulness, lower is better for latency
summary_df["score"] = (
    2.0 * summary_df["correctness_mean"]
    + 1.0 * summary_df["faithfulness_mean"]
    - 0.05 * summary_df["mean_latency_s"]
)

summary_df.sort_values("score", ascending=False)


,top_k,mean_latency_s,p95_latency_s,correctness_mean,faithfulness_mean,correctness_n,faithfulness_n,score
1,5,32.10,41.78,2.05,1.0,10,10,3.4950
0,3,21.23,28.20,1.70,1.0,10,10,3.3385
2,10,44.60,58.58,2.00,0.9,10,10,2.6700


In [27]:
def categorize(row):
    c = row["correctness_num"]
    f = row["faithfulness_num"]
    if pd.isna(c) or pd.isna(f):
        return "ungraded"
    if c >= 1 and f >= 1:
        return "good"
    if c < 1 and f >= 1:
        return "retrieval_miss_or_context_mismatch"
    if f < 1:
        return "hallucination_or_not_grounded"
    return "other"

for k, df in dfs.items():
    df["failure_type"] = df.apply(categorize, axis=1)
    print("\nTOP_K=", k)
    print(df["failure_type"].value_counts())



TOP_K= 3
failure_type
good                                  8
retrieval_miss_or_context_mismatch    2
Name: count, dtype: int64

TOP_K= 5
failure_type
good                                  7
retrieval_miss_or_context_mismatch    3
Name: count, dtype: int64

TOP_K= 10
failure_type
good                                  8
hallucination_or_not_grounded         1
retrieval_miss_or_context_mismatch    1
Name: count, dtype: int64


In [28]:
K_TO_INSPECT = summary_df.sort_values("score", ascending=False).iloc[0]["top_k"]
K_TO_INSPECT = int(K_TO_INSPECT)

dfi = dfs[K_TO_INSPECT].copy()


dfi_fail = dfi[dfi["failure_type"] != "good"][["id","question","failure_type","answer","retrieved_sources","notes"]]
dfi_fail.head(10)

,id,question,failure_type,answer,retrieved_sources,notes
3,q04,What are key diagnostic and management conside...,retrieval_miss_or_context_mismatch,"Based on the provided context, key diagnostic ...","[{""page"": 167, ""chunk_id"": 646}, {""page"": 3567...",NaN
4,q05,What are common symptoms and diagnostic approa...,retrieval_miss_or_context_mismatch,**Common symptoms of gastritis and peptic ulce...,"[{""page"": 204, ""chunk_id"": 811}, {""page"": 199,...",NaN
7,q08,What are important concepts in evaluation and ...,retrieval_miss_or_context_mismatch,"Based on the provided context, important conce...","[{""page"": 1024, ""chunk_id"": 4252}, {""page"": 10...",NaN


In [ ]:
default_TOP_K = 5